# **DATA WRANGLING LAB**

#### IMPORT REQUIRED LIBRARIES

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

---

#### LOAD DATASET

In [2]:
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

---

#### EXPLORE DATASET

In [ ]:
# Column Data Types
print(f"Data types of all columns:\n{df.dtypes}")

In [ ]:
# Counts of all columns
print(f"Counts of all columns:\n{df.count()}")

In [ ]:
# Missing values counts
print(f"Number of missing values:\n{df.isna().sum()}")

In [6]:
# Statistical summary
print("Statistical Summary:\n")
df.describe()

Statistical Summary:



,ResponseId,CompTotal,WorkExp,JobSatPoints_1,JobSatPoints_4,JobSatPoints_5,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,ConvertedCompYearly,JobSat
count,65437.000000,3.374000e+04,29658.000000,29324.000000,29393.000000,29411.000000,29450.000000,29448.00000,29456.000000,29456.000000,29450.000000,29445.000000,2.343500e+04,29126.000000
mean,32719.000000,2.963841e+145,11.466957,18.581094,7.522140,10.060857,24.343232,22.96522,20.278165,16.169432,10.955713,9.953948,8.615529e+04,6.935041
std,18890.179119,5.444117e+147,9.168709,25.966221,18.422661,21.833836,27.089360,27.01774,26.108110,24.845032,22.906263,21.775652,1.867570e+05,2.088259
min,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000
25%,16360.000000,6.000000e+04,4.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,3.271200e+04,6.000000
50%,32719.000000,1.100000e+05,9.000000,10.000000,0.000000,0.000000,20.000000,15.00000,10.000000,5.000000,0.000000,0.000000,6.500000e+04,7.000000
75%,49078.000000,2.500000e+05,16.000000,22.000000,5.000000,10.000000,30.000000,30.00000,25.000000,20.000000,10.000000,10.000000,1.079715e+05,8.000000
max,65437.000000,1.000000e+150,50.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,1.625660e+07,10.000000


---

#### Identifying Inconsistencies

In [27]:
# See how many entries there are
print(f"Number of unique countries: {df['Country'].nunique()}")

# Print all unique countries sorted alphabetically to spot typos
# We convert to list to see the full output
countries = sorted(df['Country'].dropna().unique().astype(str))

Number of unique countries: 182


#### Fix Formatting

In [8]:
# Convert everything to title case
df['Country'] = df['Country'].str.title()

# Strip whitespace (removes invisible spaces at start/end)
df['Country'] = df['Country'].str.strip()

#### Standardize entries in `Country` column

In [24]:
# Let's use pycountry libraries to create a valid country names
import pycountry

# Store valid countries names to valid_countries variable
valid_countries = {country.name for country in pycountry.countries}

# Filter out inconsistent countries name
suspects = df[~df['Country'].isin(valid_countries)]['Country'].unique()

print(f"Found {len(suspects)} non-standard country names")

Found 24 non-standard country names


In [10]:
# 1. The Dictionary of fixes
country_fix = {
    'United States Of America': 'United States',
    'United Kingdom Of Great Britain And Northern Ireland': 'United Kingdom',
    'Republic Of Korea': 'South Korea',
    'Democratic People\'S Republic Of Korea': 'North Korea',
    'Czech Republic': 'Czechia',          # Official short name update
    'Swaziland': 'Eswatini',              # Country changed its name
    'Turkey': 'Turkey',                   # Pycountry expects "Türkiye", but Turkey is fine for English analysis
    'Hong Kong (S.A.R.)': 'Hong Kong',
    'Mainland China': 'China',
    'Taiwan': 'Taiwan',                   # Pycountry often expects "Taiwan, Province of China"
    
    # Fixing the Truncated "..." entries
    'Iran, Islamic Republic Of...': 'Iran',
    'Venezuela, Bolivarian Republic Of...': 'Venezuela',
    'Micronesia, Federated States Of...': 'Micronesia',
    'Congo, Republic Of The...': 'Republic of the Congo'
}

# 2. Apply the map
df['Country'] = df['Country'].replace(country_fix)

# 3. Handle the "Not a Country" stuff
# Drop rows where country is 'Nomadic' or missing (nan)
df = df[df['Country'] != 'Nomadic']
df = df.dropna(subset=['Country'])

print("Country cleanup complete.")

Country cleanup complete.


---

#### ENCODING CATEGORICAL VARIABLES

We'll encode `employment` column using one-hot encoding

In [26]:
# The simple way to encode the 'Employment' column
# This will remove the original 'Employment' column and replace it with the new ones.
df_encoded = pd.get_dummies(df, columns=['Employment'])

---

#### HANDLING MISSING VALUES

Handle missing value in `RemoteWork` column

In [23]:
# Identify columns with most missing values
print(df.isna().sum().sort_values(ascending=False).nlargest(10))

AINextMuch less integrated       57814
AINextLess integrated            56631
AINextNo change                  46713
AINextMuch more integrated       45826
EmbeddedAdmired                  42504
EmbeddedWantToWorkWith           41662
EmbeddedHaveWorkedWith           37218
ConvertedCompYearly              35456
AIToolNot interested in Using    35268
AINextMore integrated            35090
dtype: int64


In [13]:
# Impute missing values for RemoteWork column
freq_rw = df['RemoteWork'].mode()[0]

df['RemoteWork'] = df['RemoteWork'].fillna(freq_rw)

# Verify
print(f"Number of missing Remote Work data: {df['RemoteWork'].isna().sum()}")

Number of missing Remote Work data: 0


Handle missing values in `ConvertedCompYearly` column

In [14]:
# ---Handle missing value in 'ConvertedCompYearly'---

# First let's count the missing values
print(f"Number of missing Annual Compensation data: {df['ConvertedCompYearly'].isna().sum()}")

Number of missing Annual Compensation data: 35456


Since the number of missing values is too high (more than 50%) to be replaced by median or mean. It's better to create a new clean dataframe by dropping missing values in this column.

In [15]:
comp_df = df.dropna(subset=['ConvertedCompYearly']).copy()

print(f"Number of missing values: {comp_df['ConvertedCompYearly'].isna().sum()}")
print(f"Shape of new comp dataframe: {comp_df.shape}")

Number of missing values: 0
Shape of new comp dataframe: (23431, 114)


---

#### FEATURE SCALING AND TRANSFORMATION

Before normalize the data, we'll remove any outliers first


In [16]:
# ---REMOVE OUTLIERS USING IQR---
Q1 = comp_df['ConvertedCompYearly'].quantile(0.25)
Q3 = comp_df['ConvertedCompYearly'].quantile(0.75)
IQR = Q3 - Q1

clean_comp = comp_df[(comp_df['ConvertedCompYearly'] >= Q1 - 1.5 * IQR) &
                     (comp_df['ConvertedCompYearly'] <= Q3 + 1.5 * IQR)].copy()

print(f"Shape of new clean dataframe: {clean_comp.shape}")

Shape of new clean dataframe: (22453, 114)


Apply Min-Max scaling to normalize `ConvertedCompYearly` column

In [17]:
clean_comp['ConvertedCompYearly_MinMax'] = ((clean_comp['ConvertedCompYearly'] - clean_comp['ConvertedCompYearly'].min()) /
                                            (clean_comp['ConvertedCompYearly'].max() - clean_comp['ConvertedCompYearly'].min()))

Log-transform the ConvertedCompYearly column to reduce skewness.

In [18]:
import numpy as np

# Transform to log
clean_comp['ConvertedCompYearly_Log'] = np.log1p(clean_comp['ConvertedCompYearly'])

print(f"Original Skewness: {clean_comp['ConvertedCompYearly'].skew()}")
print(f"Log Transformed Skewness: {clean_comp['ConvertedCompYearly_Log'].skew()}")

Original Skewness: 0.735462597159766
Log Transformed Skewness: -2.5160117203372248


Seems like **log transformation** is too powerful for the data so it makes the skewness worst. Let's try using **square root**.

In [19]:
clean_comp['ConvertedCompYearly_sqrt'] = np.sqrt(clean_comp['ConvertedCompYearly'])

print(f"Original Skewness: {clean_comp['ConvertedCompYearly'].skew()}")
print(f"Square root Skewness: {clean_comp['ConvertedCompYearly_sqrt'].skew()}")

Original Skewness: 0.735462597159766
Square root Skewness: -0.1828742451946956


**Square Roots** works better for this data skewness, let's keep this result.

---

#### FEATURE ENGINEERING

Create a new column `ExperienceLevel` based on the `YearsCodePro` column

In [20]:
# Convert column to numerical type
df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')

# --Define Logic---
# Bins: 0-2 (Junior), 3-5 (Mid), 6-10 (Senior), 11+ (Expert)
bins = [-1, 2, 5, 10, 100]
labels = ['Junior', 'Mid-Level', 'Senior', 'Expert']


# Create 'ExperienceLevel' column
df['ExperienceLevel'] = pd.cut(df['YearsCodePro'], bins=bins, labels=labels)

# Check the results
df[['YearsCodePro', 'ExperienceLevel']].sample(10)

,YearsCodePro,ExperienceLevel
59908,NaN,NaN
54015,6.0,Senior
56487,NaN,NaN
37326,10.0,Senior
46944,4.0,Mid-Level
45874,8.0,Senior
50533,4.0,Mid-Level
14148,NaN,NaN
15249,3.0,Mid-Level
2563,35.0,Expert


#### **Summary**

I have been practicing:

* Explored the dataset to identify inconsistencies and missing values.

* Encoded categorical variables for analysis.

* Handled missing values using imputation techniques.

* Normalized and transformed numerical data to prepare it for analysis.

* Engineered a new feature to enhance data interpretation.